In [1]:
import plotly.graph_objects as go
import plotly.io as pio
import random
import re
from datetime import datetime

import gzip
import pandas as pd
import random
import numpy as np
import warnings
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
from copy import deepcopy
import multiprocessing as mp
import tqdm

# from google.colab import drive
import os

# drive.mount('/content/drive')



warnings.simplefilter("ignore")


# Data Processing

### Extract Compressed Weather Data

In [2]:
import zipfile
with zipfile.ZipFile("./weatherdata.zip", 'r') as zip_ref:
    zip_ref.extractall(".")

### Load Data (Generating Job Queue)

In [ ]:
"""CHECK THIS BEFORE RUNNING DATA COLLECTION

interval = 5 was used for data collection for training final IAE model
"""

interval = 20 # Step size in degrees lat/long when generating the job queue (Higher = less computation time)


In [2]:
folder_path = "./weatherdata/"
random.seed(0)

files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) and f.endswith('04I.txt.gz')]
sorted_file_list = sorted(files)



# Generate job queue for parallelization
# Each job is responsible for a 5x5 lat/long square on the globe
def gen_job_queue(sorted_file_list):
    job_buckets = []
    job_num = 0
    interval = 20
    box_size = 5
    bucket_index = 0
    for min_lat in np.arange(-59.75, 54.95, interval): #90, 20)):
        max_lat = min_lat + box_size
        if max_lat > 90:
            continue
            
        for min_lon in np.arange(-179.95, 179.95, interval): #180, 20)):
            max_lon = min_lon + box_size
            if max_lon > 180:
                continue
                
            job_list = []
            for i,file_name in enumerate(sorted_file_list):
                job_list.append((np.round(min_lat,2), np.round(max_lat,2), np.round(min_lon,2), np.round(max_lon,2), file_name, i, job_num, bucket_index))
                job_num += 1
                
            job_buckets.append(job_list)
            bucket_index += 1
            
    return job_buckets

jobQ = gen_job_queue(sorted_file_list)





### Input how many threads you want to process the data at a time

In [3]:
NUM_THREADS = 31 # IF THIS NUMBER IS CLOSE TO YOUR LOGICAL PROCESSOR COUNT THEN EXPECT EXTREME SLOWDOWN ON YOUR COMPUTER

In [4]:
from job_functions import parse_file, make_trios

def collect_all_data(jobQ):
    trios = []
    for bucket in tqdm.tqdm(jobQ):
        L = []
        with mp.Pool(NUM_THREADS) as pool:
            L = pool.map(parse_file, bucket)
        new_trios = make_trios(L)

        # Filter out jobs that were excluded because there wasn't enough data for that square
        new_trios = list(filter(lambda item: len(item.shape) != 1, new_trios))
        trios.extend(new_trios)
    return trios

        
DATA = collect_all_data(jobQ)

100%|██████████| 108/108 [06:54<00:00,  3.84s/it]


## Save data to numpy file for use in model

In [1]:
with open('data.npy', 'wb') as f:
    np.save(f, np.array(DATA))

NameError: name 'np' is not defined